In [ ]:

# هذا مثال للاستخدام في جوبيتر نوت بوك
import sys
sys.path.append('..')

from src.preprocessing import ArabicTextPreprocessor
from src.bert_embeddings import BERTArabicEmbeddings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


In [ ]:

# 1. تحميل البيانات
with open('../data/raw/al_fatiha.txt', 'r', encoding='utf-8') as f:
    verses = f.readlines()


In [ ]:

# 2. المعالجة المسبقة
preprocessor = ArabicTextPreprocessor(remove_diacritics=True)
cleaned_verses = []
for verse in verses:
    cleaned, _ = preprocessor.clean_text(verse)
    cleaned_verses.append(cleaned)

# 3. تحميل BERT
bert_extractor = BERTArabicEmbeddings()

# 4. استخراج التضمينات
embeddings_data = bert_extractor.get_contextual_embeddings(cleaned_verses)

# 5. تحليل
print(f"شكل تضمينات الآيات: {embeddings_data['verse_embeddings'].shape}")
print(f"معلومات النموذج: {embeddings_data['model_info']}")

# 6. تصور باستخدام PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings_data['verse_embeddings'])

plt.figure(figsize=(10, 8))
for i, (x, y) in enumerate(reduced_embeddings):
    plt.scatter(x, y, s=200, alpha=0.7)
    plt.text(x, y, f'آية {i+1}', fontsize=12, ha='center', va='center')
    
plt.title('تصور تضمينات BERT لآيات سورة الفاتحة (PCA)', fontsize=14)
plt.xlabel('المكون الرئيسي الأول')
plt.ylabel('المكون الرئيسي الثاني')
plt.grid(True, alpha=0.3)
plt.show()

# 7. حساب التشابه بين الآيات
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(embeddings_data['verse_embeddings'])

plt.figure(figsize=(8, 6))
plt.imshow(similarity_matrix, cmap='hot', interpolation='nearest')
plt.colorbar(label='مقياس التشابه')
plt.xticks(range(len(verses)), [f'آية {i+1}' for i in range(len(verses))], rotation=45)
plt.yticks(range(len(verses)), [f'آية {i+1}' for i in range(len(verses))])
plt.title('مصفوفة التشابه بين الآيات (BERT)')
plt.tight_layout()
plt.show()
